# Mobilenetv2 Object detection using Custom dataset (In Progress)

**Note: the code is still in progress and is subjected to change**

The code below covers the following

- setting up the environment with all prerequisites packages that are needed to be installed for the API to work efficiently
- The preprocessing of the image dataset
- generation of tf.records
- generation of csv annotation file 
- Configuring the model with respect to the custom dataset
- Training of custom model 
- generating of inference graph of the trained model

## **colab prerequisites and mounting it to drive**

In [ ]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [ ]:
!mkdir -p drive
!google-drive-ocamlfuse drive

/bin/bash: google-drive-ocamlfuse: command not found


## **Change directory**

In [ ]:
cd /content/drive/My Drive/Final_Model/Mobilenet/models_Tf/research/object_detection/

/content/drive/My Drive/Final_Model/Mobilenet/models_Tf/research/object_detection


In [ ]:
pwd

'/content/drive/My Drive/Final_Model/Mobilenet/models_Tf/research/object_detection'

In [ ]:
cd /content/drive/My Drive/Final_Model/Mobilenet/models_Tf/research/

/content/drive/My Drive/Final_Model/Mobilenet/models_Tf/research


## **check tensorflow version**

In [ ]:
! python -c 'import tensorflow as tf; print(tf.__version__)' 

2020-07-01 05:14:19.191531: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2.2.0


In [ ]:
import tensorflow as tf
tf.test.is_gpu_available()
#tf.config.list_physical_devices('GPU')

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

## **Changing tensorflow version from 2.0.0 to 1.15.2**

In [ ]:
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)

TensorFlow 1.x selected.
1.15.2


## **Cloning the tensorflow repository**

In [ ]:
! git clone https://github.com/osman-95/models.git


Cloning into 'models'...
remote: Enumerating objects: 35066, done.
remote: Total 35066 (delta 0), reused 0 (delta 0), pack-reused 35066
Receiving objects: 100% (35066/35066), 512.86 MiB | 18.92 MiB/s, done.
Resolving deltas: 100% (22765/22765), done.
Checking out files: 100% (2512/2512), done.


## **install prerequisites libraries**

In [ ]:
! pip install --user Cython
! pip install --user contextlib2
! pip install --user pillow
! pip install --user lxml
! pip install --user jupyter
! pip install --user matplotlib

## Installing cocoapi prerequisites

In [ ]:
#@title 
! pip install "git+https://github.com/philferriere/cocoapi.git#egg=pycocotools&subdirectory=PythonAPI"

## **Working Proto convertion code**
### **(converting the protos files to py)**

In [ ]:
! protoc --python_out=. ./object_detection/protos/*.proto  

object_detection/protos/input_reader.proto: warning: Import object_detection/protos/image_resizer.proto but not used.


In [ ]:
! protoc --python_out=. ./object_detection/protos/anchor_generator.proto 

## **Converting xml files of image annotations to csv files**

In [ ]:
! python xml_to_csv.py

Successfully converted xml to csv.


In [ ]:
! python test_xml_to_csv.py

Successfully converted xml to csv.


In [ ]:
cd /content/drive/My Drive/Final_Model/Mobilenet/models_Tf/research/

/content/drive/My Drive/Final_Model/Mobilenet/models_Tf/research


## **Building object detection prerequisites**

In [ ]:
# From within TensorFlow/models/research/
! python setup.py build
! python setup.py install

In [ ]:
import object_detection

In [ ]:
cd /content/drive/My Drive/Final_Model/Mobilenet/models_Tf/research/object_detection/

/content/drive/My Drive/Final_Model/Mobilenet/models_Tf/research/object_detection


## **TF records for train and test (first set of images (600))**

In [ ]:
! python generate_tfrecord.py --csv_input=images/Train_annot_labels.csv  --image_dir=images/train --output_path=images/train.record



W0629 21:06:55.941634 139921927149440 module_wrapper.py:139] From generate_tfrecord.py:134: The name tf.python_io.TFRecordWriter is deprecated. Please use tf.io.TFRecordWriter instead.


W0629 21:06:56.106540 139921927149440 module_wrapper.py:139] From generate_tfrecord.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

Successfully created the TFRecords: /content/drive/My Drive/Final_Model/Mobilenet/models_Tf/research/object_detection/images/train.record


In [ ]:

! python generate_tfrecord.py --csv_input=images/Test_annot_labels.csv  --image_dir=images/test --output_path=images/test.record



W0629 07:58:28.836865 140529563744128 module_wrapper.py:139] From generate_tfrecord.py:134: The name tf.python_io.TFRecordWriter is deprecated. Please use tf.io.TFRecordWriter instead.


W0629 07:58:28.914928 140529563744128 module_wrapper.py:139] From generate_tfrecord.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

Successfully created the TFRecords: /content/drive/My Drive/Final_Model/Mobilenet/models_Tf/research/object_detection/images/test.record


## **TF records for train and test (Second set of images (1500))**

In [ ]:
! python generate_tfrecord.py --csv_input=images_extra/Train_annot_labels.csv  --image_dir=images_extra/train --output_path=images_extra/train.record



W0701 23:47:46.310220 139705979041664 module_wrapper.py:139] From generate_tfrecord.py:134: The name tf.python_io.TFRecordWriter is deprecated. Please use tf.io.TFRecordWriter instead.


W0701 23:47:46.728677 139705979041664 module_wrapper.py:139] From generate_tfrecord.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

Successfully created the TFRecords: /content/drive/My Drive/Final_Model/Mobilenet/models_Tf/research/object_detection/images_extra/train.record


In [ ]:
! python generate_tfrecord.py --csv_input=images_extra/Test_annot_labels.csv  --image_dir=images_extra/test --output_path=images_extra/test.record



W0701 23:08:39.287793 139941674121088 module_wrapper.py:139] From generate_tfrecord.py:134: The name tf.python_io.TFRecordWriter is deprecated. Please use tf.io.TFRecordWriter instead.


W0701 23:08:39.366583 139941674121088 module_wrapper.py:139] From generate_tfrecord.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

Successfully created the TFRecords: /content/drive/My Drive/Final_Model/Mobilenet/models_Tf/research/object_detection/images_extra/test.record


In [ ]:
cd /content/drive/My Drive/Final_Model/Mobilenet/models_Tf/research/object_detection/

/content/drive/My Drive/Final_Model/Mobilenet/models_Tf/research/object_detection


## **setting python path** 

In [ ]:

import os
os.environ['PYTHONPATH'] += ':/content/drive/My Drive/Project_tasks/Mobilenet/models/research/:/content/drive/My Drive/Project_tasks/Mobilenet/models/research/slim/'

## **Training of the object detection model with our custom images**

In [ ]:
! python train.py --logtostderr --train_dir=training/ --pipeline_config_path=training/ssd_mobilenet_v2_coco.config

In [ ]:
! python train.py --logtostderr --train_dir=training_extra/ --pipeline_config_path=training_extra/ssd_mobilenet_v2_coco.config

In [ ]:
! python train.py --logtostderr --train_dir=training_fresh/ --pipeline_config_path=training_fresh/ssd_mobilenet_v2_coco.config

In [ ]:
! python train.py --logtostderr --train_dir=training_23/ --pipeline_config_path=training_23/ssd_mobilenet_v2_coco.config

## **Evaluate the model**

In [ ]:
! python eval.py --logtostderr --checkpoint_dir=training_fresh/ --eval_dir=eval/
  --pipelineline_config_path=training_fresh/ssd_mobilenet_v2_coco.config

## **Exporting the inference graph**

In [ ]:
! python export_inference_graph.py --input_type image_tensor --pipeline_config_path training/ssd_mobilenet_v2_coco.config --trained_checkpoint_prefix training/model.ckpt-20000 --output_directory inference_graph

In [ ]:
! python export_inference_graph.py --input_type image_tensor --pipeline_config_path training_extra/ssd_mobilenet_v2_coco.config --trained_checkpoint_prefix training_extra/model.ckpt-20000 --output_directory inference_graph_extra

In [ ]:
! python export_inference_graph.py --input_type image_tensor --pipeline_config_path training_fresh/ssd_mobilenet_v2_coco.config --trained_checkpoint_prefix training_fresh/model.ckpt-100000 --output_directory inference_graph_fresh

In [ ]:
! python export_inference_graph.py --input_type image_tensor --pipeline_config_path training_23/ssd_mobilenet_v2_coco.config --trained_checkpoint_prefix training_23/model.ckpt-100000 --output_directory inference_graph_23